* Criado por: Gabriel Quintella
* Data Criação: 24.01.2023
* Data Última Modificação: 01.02.2023
* Repositório: https://github.com/dbaassists/Projeto-KB-Spark/blob/main/Projeto%20KB%20Spark.ipynb

DESCRIÇÃO DO PROJETO

Criar uma base de conhecimento (KB) com comandos pyspark.

*  Dia 1 - 24.01.2023 - Lançamento do Projeto;
*  Dia 2 - 26.01.2023 - Aprimoramento do Projeto com melhoria nos exemplos criados;
*  Dia 3 - 27.01.2023 - Aprimoramento do Projeto com melhoria nos exemplos criados;
*  Dia 4 - 28.01.2023 - Aprimoramento do Projeto com melhoria nos exemplos criados;
*  Dia 5 - 29.01.2023 - Aprimoramento do Projeto com melhoria nos exemplos criados;
*  Dia 6 - 30.01.2023 - Aprimoramento do Projeto com melhoria nos exemplos criados;
*  Dia 7 - 31.01.2023 - Aprimoramento do Projeto com melhoria nos exemplos criados;
*  Dia 8 - 01.02.2023 - Reorganização dos tópicos e inclusões de novos exemplos;


 Sumário

* 01 - Instalação da Biblioteca PySpark
* 02 - Definição de Bibliotecas
* 02.1 - Arquivos Usados
* 03 - Métodos de Importação de Arquivos CSV
* 03.1 - Importando Arquivo CSV de forma FULL
* 03.2 - Importando Arquivo CSV Definindo o Delimitador de Colunas
* 03.2.1 - Comando Semelhante ao DataFrame.info() do Pandas
* 03.2.2 - Consultando a Estrutura do DataFrame Spark
* 03.3 - Importando Arquivo CSV Definindo Delimitador de Colunas e Cabeçalho
* 03.4 - Importando TODOS os Arquivos CSV de um Diretório
* 03.5 - Importando Arquivo CSV Definindo inferSchema
* 03.6 - Importando Arquivo CSV Definindo encoding
* 03.7 - Definindo a Estrutura de um DataFrame e Importar CSV
* 04 - Trabalhando com Datas - Converte coluna String (DD/MM/AAA) para Date (AAAAMMDD)
* 04.1 - Trabalhando com Datas - A partir de uma coluna de Data, criar outra de Ano
* 04.2 - Trabalhando com Datas - A partir de uma coluna de Data, criar outra de Mês
* 04.3 - Trabalhando com Datas - A partir de uma coluna de Data, criar outra de Dia
* 05 - Manipulando informações de um DataFrame
* 05.1 - Tratando Valores Nulos
* 05.2 - Filtrando Valores
* 05.2.1 - Semelhante a cláusula LIKE - "CONTÉM" de Bancos de Dados
* 05.2.2 - Semelhante a cláusula LIKE - "INICIA COM" de Bancos de Dados
* 05.2.3 - Semelhante a cláusula LIKE - "TERMINA COM" de Bancos de Dados
* 05.2.4 - Semelhante a cláusula WHERE de Bancos de Dados
* 05.2.5 - Semelhante a cláusula IN de Bancos de Dados
* 05.3 - Tratando Valores com UPPER, LOWER e INITCAPT
* 06 - Transformando o DataFrame Spark em Tabela Temporária
* 06.1 - Lendo a Tabela e Transformando em outro DataFrame Spark
* 07 - Convertendo DataType de um DataFrame Spark
* 07.1 - De uma Coluna
* 07.2 - Criando uma nova Coluna
* 08 - Renomeando Coluna(s) em DataFrame Spark
* 08.1 - Uma Única Coluna
* 08.2 - Várias Colunas
* 09 - Ordenando um DataFrame Spark
* 09.1 - Ordenando um DataFrame Spark por uma Coluna
* 09.2 - Ordenando um DataFrame Spark por Várias Colunas
* 10 - Agrupando um DataFrame Spark
* 11 - Funções de Agrupamento (Count(), Sum(), Min(), Max() e Mean())

# 01 - Instalação da Biblioteca PySpark

In [1]:
#pip install pyspark

# 02 - Definição de Bibliotecas

In [2]:
import sys
import requests
import pandas as pd
import json
import pyspark.sql.functions as F #import explode_outer, col
from pyspark.sql.types import *
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkConf
from pyspark import SparkContext
import urllib
from sqlalchemy import __version__ as sa_version, create_engine, text


# 02.1 - Arquivos Usados

In [3]:
arquivoDadosEleitorais = '../ProjetoSparkKB/Dados/Candidato/consulta_cand_2022_BRASIL.csv'

arquivoIndicadorHomicidio = '../ProjetoSparkKB/Dados/IndicadorSegurancaPublica/indicador_homocidio.csv'

arquivoIndicadorHomicidio2 = '../ProjetoSparkKB/Dados/IndicadorSegurancaPublica/indicador_homocidio_2.csv'

arquivoIndicadorHomicidioRes = '../ProjetoSparkKB/Dados/IndicadorSegurancaPublica/indicador_homocidio_res.csv'

In [4]:
spark = SparkSession.builder.getOrCreate()

# 03 - Métodos de Importação de Arquivos CSV

* A CSV dataset is pointed to by path.
* The path can be either a single CSV file or a directory of CSV files
* path = "examples/src/main/resources/people.csv"

df = spark.read.csv(path)

df.show()

** Documentação: https://spark.apache.org/docs/latest/sql-data-sources-csv.html

# 03.1 - Importando Arquivo CSV de forma FULL

In [5]:

df = spark.read.csv(arquivoDadosEleitorais)


# 03.2 - Importando Arquivo CSV Definindo o Delimitador de Colunas

* Read a csv with delimiter, the default delimiter is ","

In [6]:


df = spark.read.option('delimiter',';').csv(arquivoDadosEleitorais)


## 03.2.1 - Comando Semelhante ao DataFrame.info() do Pandas

In [7]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (nullable = tru

## 03.2.2 - Consultando a Estrutura do DataFrame Spark

In [8]:
df.summary().show()

+-------+----------+----------+-----------+---------------+-----------------+--------------------+-------------------+--------------------+----------+--------------+-----+-----+---------+------------------+---------------+--------------------+------------------+--------------------+-------------+----------+--------------------+--------------+--------------------+------+--------------------+---------+-------------+-----------------+-----+------------+-------------------+------------+------------+-------------+--------------------+--------------------+------+------------------+--------------------+-----+--------------------+-------------------+-------------+-------------------+--------------------+------------------+--------------+------------------+-------------------+-----------------+---------+------------------+-------+------------------+-------------+--------------------+------------------+--------+------------+----------------+--------------------+--------------------+-------------

In [9]:
#df.write.option("header",True) \
# .csv("/tmp/spark_output/zipcodes")

# Entendendo a diferença entre as funções option() e options()

# 03.3 - Importando Arquivo CSV Definindo Delimitador de Colunas e Cabeçalho

## Usando Função: option()

* Read a csv with delimiter and a header

In [10]:


df = spark.read.option('delimiter',';') \
                .option('header','True') \
                .csv(arquivoDadosEleitorais)

#df.select('DT_GERACAO').show()

df.show(2,truncate=False)

+----------+----------+-----------+---------------+-----------------+--------+----------+------------------------------+----------+--------------+-----+-----+--------------+--------+-----------------+------------+------------+---------------------+---------------------+-------------------+----------------+--------------+-----------------------+-----------------------+------------------------+------------------------+---------------+----------+-------------+------------------------------+------------+-------------------+------------+-----------------------+------------+---------------+-----------------------+----------------+----------------+----------------+-----------------------+-----------------------+-------------+-------------------+-----------------------------+---------+---------+-----------------+-------------------+---------------+---------------+-----------+-----------+-----------+-------------------------+-----------------------+----------------+----------------+------------

## Usando Função: options()

* You can also use options() to use multiple options

In [11]:


df = spark.read.options(delimiter=';'
                        , header='True') \
                        .csv(arquivoDadosEleitorais)

#df.select('DT_GERACAO').show()

df.show(2,truncate=False)

+----------+----------+-----------+---------------+-----------------+--------+----------+------------------------------+----------+--------------+-----+-----+--------------+--------+-----------------+------------+------------+---------------------+---------------------+-------------------+----------------+--------------+-----------------------+-----------------------+------------------------+------------------------+---------------+----------+-------------+------------------------------+------------+-------------------+------------+-----------------------+------------+---------------+-----------------------+----------------+----------------+----------------+-----------------------+-----------------------+-------------+-------------------+-----------------------------+---------+---------+-----------------+-------------------+---------------+---------------+-----------+-----------+-----------+-------------------------+-----------------------+----------------+----------------+------------

# 03.4 - Importando TODOS os Arquivos CSV de um Diretório

## Usando spark.read.csv()

* Read all files in a folder, please make sure only CSV files should present in the folder.

In [12]:

folderPath = "../ProjetoSparkKB/Dados/Candidato/"
df5 = spark.read.csv(folderPath,sep=';')
df5.count()

62204

## Usando spark.read.format()


In [13]:

dfExemplo2 = spark.read.format("csv").load(arquivoDadosEleitorais)
    
dfExemplo2.printSchema()

root
 |-- _c0: string (nullable = true)



# 03.5 - Importando Arquivo CSV Definindo inferSchema

## Entendendo o parâmetro inferSchema

## Parâmetro inferSchema FALSE

In [14]:

dfExemplo3 = spark.read.option("inferSchema",False) \
                .option("delimiter",";") \
                .option("header",True) \
                .csv(arquivoDadosEleitorais)

dfExemplo3.printSchema()

root
 |-- DT_GERACAO: string (nullable = true)
 |-- HH_GERACAO: string (nullable = true)
 |-- ANO_ELEICAO: string (nullable = true)
 |-- CD_TIPO_ELEICAO: string (nullable = true)
 |-- NM_TIPO_ELEICAO: string (nullable = true)
 |-- NR_TURNO: string (nullable = true)
 |-- CD_ELEICAO: string (nullable = true)
 |-- DS_ELEICAO: string (nullable = true)
 |-- DT_ELEICAO: string (nullable = true)
 |-- TP_ABRANGENCIA: string (nullable = true)
 |-- SG_UF: string (nullable = true)
 |-- SG_UE: string (nullable = true)
 |-- NM_UE: string (nullable = true)
 |-- CD_CARGO: string (nullable = true)
 |-- DS_CARGO: string (nullable = true)
 |-- SQ_CANDIDATO: string (nullable = true)
 |-- NR_CANDIDATO: string (nullable = true)
 |-- NM_CANDIDATO: string (nullable = true)
 |-- NM_URNA_CANDIDATO: string (nullable = true)
 |-- NM_SOCIAL_CANDIDATO: string (nullable = true)
 |-- NR_CPF_CANDIDATO: string (nullable = true)
 |-- NM_EMAIL: string (nullable = true)
 |-- CD_SITUACAO_CANDIDATURA: string (nullable = tr

## Parâmetro inferSchema TRUE

In [15]:

dfExemplo04 = spark.read.options(header='True'
                                 , inferSchema='True'
                                 , delimiter=';') \
                                 .csv(arquivoDadosEleitorais)

dfExemplo04.printSchema()

root
 |-- DT_GERACAO: string (nullable = true)
 |-- HH_GERACAO: timestamp (nullable = true)
 |-- ANO_ELEICAO: integer (nullable = true)
 |-- CD_TIPO_ELEICAO: integer (nullable = true)
 |-- NM_TIPO_ELEICAO: string (nullable = true)
 |-- NR_TURNO: integer (nullable = true)
 |-- CD_ELEICAO: integer (nullable = true)
 |-- DS_ELEICAO: string (nullable = true)
 |-- DT_ELEICAO: string (nullable = true)
 |-- TP_ABRANGENCIA: string (nullable = true)
 |-- SG_UF: string (nullable = true)
 |-- SG_UE: string (nullable = true)
 |-- NM_UE: string (nullable = true)
 |-- CD_CARGO: integer (nullable = true)
 |-- DS_CARGO: string (nullable = true)
 |-- SQ_CANDIDATO: long (nullable = true)
 |-- NR_CANDIDATO: integer (nullable = true)
 |-- NM_CANDIDATO: string (nullable = true)
 |-- NM_URNA_CANDIDATO: string (nullable = true)
 |-- NM_SOCIAL_CANDIDATO: string (nullable = true)
 |-- NR_CPF_CANDIDATO: long (nullable = true)
 |-- NM_EMAIL: string (nullable = true)
 |-- CD_SITUACAO_CANDIDATURA: integer (nullabl

# 03.6 - Importando Arquivo CSV Definindo encoding

## Entendendo o parâmetro encoding

In [16]:

dfExemplo04 = spark.read.options(header='True'
                                 , inferSchema='True'
                                 , delimiter=';'
                                 , encoding="windows-1252") \
                                 .csv(arquivoDadosEleitorais)

dfExemplo04.show()


+----------+-------------------+-----------+---------------+-----------------+--------+----------+--------------------+----------+--------------+-----+-----+-----------------+--------+-----------------+------------+------------+--------------------+--------------------+-------------------+----------------+--------------+-----------------------+-----------------------+------------------------+------------------------+---------------+----------+-------------+--------------------+------------+--------------------+--------------+-----------------------+------------+---------------+-----------------------+----------------+----------------+----------------+-----------------------+-----------------------+-------------+-------------------+-----------------------------+---------+---------+-----------------+--------------------+---------------+---------------+-----------+-----------+-----------+--------------------+-----------------------+----------------+----------------+------------+----------

# 03.7 - Definindo a Estrutura de um DataFrame e Importar CSV

** Documentação: https://sqlrelease.com/spark-read-file-with-special-characters-using-pyspark

** Documentação: https://sparkbyexamples.com/pyspark/pyspark-sql-types-datatype-with-examples/

** Documentação: https://spark.apache.org/docs/latest/sql-ref-datatypes.html

** Documentação: https://sparkbyexamples.com/pyspark/pyspark-date_format-convert-date-to-string-format/

### O arquivo "indicador_homocidio.csv" foi criado com a coluna Data do tipo "Data"
### O que permitiu a criação do DataFrame com a coluna Data do tipo DateType()

In [17]:

schema = StructType() \
      .add("CodUf",StringType(),True) \
      .add("TipoCrime",StringType(),True) \
      .add("Quantidade",IntegerType(),True) \
      .add("Data",DateType(),True)


df_with_schema = spark.read.options(header='True'
                                    , inferSchema='False'
                                    , delimiter=';'
                                    , encoding="windows-1252") \
                                    .schema(schema) \
                                    .csv(arquivoIndicadorHomicidio)

df_with_schema.printSchema()

df_with_schema.show()

root
 |-- CodUf: string (nullable = true)
 |-- TipoCrime: string (nullable = true)
 |-- Quantidade: integer (nullable = true)
 |-- Data: date (nullable = true)

+-------------------+----------------+----------+----------+
|              CodUf|       TipoCrime|Quantidade|      Data|
+-------------------+----------------+----------+----------+
|            Roraima|Homicídio doloso|      1389|2023-01-28|
|              Amapá|Homicídio doloso|      1948|2023-01-29|
|               Acre|Homicídio doloso|      2007|2023-01-30|
|          Tocantins|Homicídio doloso|      2787|2023-01-31|
|           Rondônia|Homicídio doloso|      3482|2023-02-01|
|   Distrito Federal|Homicídio doloso|      3518|2023-02-02|
| Mato Grosso do Sul|Homicídio doloso|      3905|2023-02-03|
|              Piauí|Homicídio doloso|      4714|2023-02-04|
|     Santa Catarina|Homicídio doloso|      5832|2023-02-05|
|        Mato Grosso|Homicídio doloso|      6920|2023-02-06|
|            Sergipe|Homicídio doloso|      69

# 04 - Trabalhando com Datas - Converte coluna String (DD/MM/AAA) para Date (AAAAMMDD)

### O arquivo "indicador_homocidio_2.csv" foi criado com a coluna Data do tipo "Geral"
### Nesse caso, vamos precisar criar o DataFrame com a coluna Data do tipo StringType() e em seguida realizar a conversão da coluna para o tipo Date

* Trabalhando com Datas

** Documentação: https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html

In [18]:
from pyspark.sql.functions import *
from pyspark.sql.types import StringType

schema = StructType() \
      .add("CodUf",StringType(),True) \
      .add("TipoCrime",StringType(),True) \
      .add("Quantidade",IntegerType(),True) \
      .add("Data",StringType(),True)

df_with_schema2 = spark.read.options(header='True', inferSchema='False', delimiter=';', encoding="windows-1252") \
      .schema(schema) \
      .csv(arquivoIndicadorHomicidio2)

df_with_schema2.show()

print('------------------------------ ESTRUTURA ANTES DA CONVERSÃO ------------------------------')

df_with_schema2.printSchema()

print('------------------------------------------------------------------------------------------')

df_with_schema2 = df_with_schema2.withColumn('Data', to_date(date_format(unix_timestamp(df_with_schema2.Data , "dd/mm/yyyy").cast("timestamp"),"yyyy-mm-dd")))

print('------------------------------ ESTRUTURA DEPOIS DA CONVERSÃO ------------------------------')

df_with_schema2.printSchema()

df_with_schema2.show()

+-------------------+----------------+----------+----------+
|              CodUf|       TipoCrime|Quantidade|      Data|
+-------------------+----------------+----------+----------+
|            Roraima|Homicídio doloso|      1389|28/01/2023|
|              Amapá|Homicídio doloso|      1948|29/01/2023|
|               Acre|Homicídio doloso|      2007|30/01/2023|
|          Tocantins|Homicídio doloso|      2787|31/01/2023|
|           Rondônia|Homicídio doloso|      3482|01/02/2023|
|   Distrito Federal|Homicídio doloso|      3518|02/02/2023|
| Mato Grosso do Sul|Homicídio doloso|      3905|03/02/2023|
|              Piauí|Homicídio doloso|      4714|04/02/2023|
|     Santa Catarina|Homicídio doloso|      5832|05/02/2023|
|        Mato Grosso|Homicídio doloso|      6920|06/02/2023|
|            Sergipe|Homicídio doloso|      6970|07/02/2023|
|           Amazonas|Homicídio doloso|      8590|08/02/2023|
|     Espírito Santo|Homicídio doloso|      8695|09/02/2023|
|            Paraíba|Hom

# 04.1 - Trabalhando com Datas - A partir de uma coluna de Data, criar outra de Ano

In [19]:

df_with_schema2 = df_with_schema2.withColumn("Ano", date_format(col("Data"), "y"))


df_with_schema2.printSchema()

root
 |-- CodUf: string (nullable = true)
 |-- TipoCrime: string (nullable = true)
 |-- Quantidade: integer (nullable = true)
 |-- Data: date (nullable = true)
 |-- Ano: string (nullable = true)



# 04.2 - Trabalhando com Datas - A partir de uma coluna de Data, criar outra de Mês

In [20]:

df_with_schema2 = df_with_schema2.withColumn("NumMes", date_format(col("Data"), "M"))

df_with_schema2 = df_with_schema2.withColumn("NomMes", date_format(col("Data"), "L"))


df_with_schema2.printSchema()

df_with_schema2.show()

root
 |-- CodUf: string (nullable = true)
 |-- TipoCrime: string (nullable = true)
 |-- Quantidade: integer (nullable = true)
 |-- Data: date (nullable = true)
 |-- Ano: string (nullable = true)
 |-- NumMes: string (nullable = true)
 |-- NomMes: string (nullable = true)

+-------------------+----------------+----------+----------+----+------+------+
|              CodUf|       TipoCrime|Quantidade|      Data| Ano|NumMes|NomMes|
+-------------------+----------------+----------+----------+----+------+------+
|            Roraima|Homicídio doloso|      1389|2023-01-28|2023|     1|     1|
|              Amapá|Homicídio doloso|      1948|2023-01-29|2023|     1|     1|
|               Acre|Homicídio doloso|      2007|2023-01-30|2023|     1|     1|
|          Tocantins|Homicídio doloso|      2787|2023-01-31|2023|     1|     1|
|           Rondônia|Homicídio doloso|      3482|2023-02-01|2023|     2|     2|
|   Distrito Federal|Homicídio doloso|      3518|2023-02-02|2023|     2|     2|
| Mato G

# 04.3 - Trabalhando com Datas - A partir de uma coluna de Data, criar outra de Dia

In [21]:

df_with_schema2 = df_with_schema2.withColumn("NumDiaAno", date_format(col("Data"), "D"))

df_with_schema2 = df_with_schema2.withColumn("NumDiaMes", date_format(col("Data"), "d"))

df_with_schema2.printSchema()

df_with_schema2.show()

root
 |-- CodUf: string (nullable = true)
 |-- TipoCrime: string (nullable = true)
 |-- Quantidade: integer (nullable = true)
 |-- Data: date (nullable = true)
 |-- Ano: string (nullable = true)
 |-- NumMes: string (nullable = true)
 |-- NomMes: string (nullable = true)
 |-- NumDiaAno: string (nullable = true)
 |-- NumDiaMes: string (nullable = true)

+-------------------+----------------+----------+----------+----+------+------+---------+---------+
|              CodUf|       TipoCrime|Quantidade|      Data| Ano|NumMes|NomMes|NumDiaAno|NumDiaMes|
+-------------------+----------------+----------+----------+----+------+------+---------+---------+
|            Roraima|Homicídio doloso|      1389|2023-01-28|2023|     1|     1|       28|       28|
|              Amapá|Homicídio doloso|      1948|2023-01-29|2023|     1|     1|       29|       29|
|               Acre|Homicídio doloso|      2007|2023-01-30|2023|     1|     1|       30|       30|
|          Tocantins|Homicídio doloso|      27

# 05 - Manipulando informações de um DataFrame

## 05.1 - Tratando Valores Nulos

In [22]:

schema = StructType() \
      .add("CodUf",StringType(),True) \
      .add("TipoCrime",StringType(),True) \
      .add("Quantidade",IntegerType(),True) \
      .add("Data",StringType(),True) \


df_with_schema = spark.read.options(header='True'
                                    , inferSchema='False'
                                    , delimiter=';'
                                    , encoding="windows-1252") \
                                    .schema(schema) \
                                    .csv(arquivoIndicadorHomicidioRes)

#df_with_schema = df_with_schema.fillna(0) # ==> NÃO FUNCIONA PARA A COLUNA DATA 

#df_with_schema = df_with_schema.fillna('0') # ==> NÃO FUNCIONA PARA A COLUNA QUANTIDADE 

#df_with_schema = df_with_schema.na.fill(0) # ==> NÃO FUNCIONA PARA A COLUNA DATA

#df_with_schema = df_with_schema.na.fill('0') # ==> NÃO FUNCIONA PARA A COLUNA QUANTIDADE 

#df_with_schema = df_with_schema.na.fill('0', ["Data"]) # ==> FUNCIONA

# df_with_schema = df_with_schema.na.fill(0, ["Data"]) # ==> NÃO FUNCIONA DEVIDO AO TIPO DE DADO

df_with_schema.printSchema()

df_with_schema.show()

root
 |-- CodUf: string (nullable = true)
 |-- TipoCrime: string (nullable = true)
 |-- Quantidade: integer (nullable = true)
 |-- Data: string (nullable = true)

+-------------------+----------------+----------+----------+
|              CodUf|       TipoCrime|Quantidade|      Data|
+-------------------+----------------+----------+----------+
|            Roraima|Homicídio doloso|      1389|28/01/2023|
|              Amapá|Homicídio doloso|      1948|29/01/2023|
|               Acre|Homicídio doloso|      2007|30/01/2023|
|          Tocantins|Homicídio doloso|      null|31/01/2023|
|           Rondônia|Homicídio doloso|      3482|01/02/2023|
|   Distrito Federal|Homicídio doloso|      3518|02/02/2023|
| Mato Grosso do Sul|Homicídio doloso|      3905|03/02/2023|
|              Piauí|Homicídio doloso|      4714|04/02/2023|
|     Santa Catarina|Homicídio doloso|      5832|05/02/2023|
|        Mato Grosso|Homicídio doloso|      6920|06/02/2023|
|            Sergipe|Homicídio doloso|      

## 05.2 - Filtrando Valores 

### 05.2.1 - Semelhante a cláusula LIKE - "CONTÉM" de Bancos de Dados

In [23]:

schema = StructType() \
      .add("CodUf",StringType(),True) \
      .add("TipoCrime",StringType(),True) \
      .add("Quantidade",IntegerType(),True) \
      .add("Data",StringType(),True)


df_with_schema = spark.read.options(header='True'
                                    , inferSchema='False'
                                    , delimiter=';'
                                    , encoding="windows-1252") \
                                    .schema(schema) \
                                    .csv(arquivoIndicadorHomicidioRes)

#df_with_schema = df_with_schema.filter(df_with_schema['CodUf'].contains('P'))


## CONTÉM
df_with_schema.filter(df_with_schema.CodUf.like("%Ri%")).show()

## INICIA COM 
df_with_schema.filter(df_with_schema.CodUf.like("Ri%")).show()

## TERMINA COM
df_with_schema.filter(df_with_schema.CodUf.like("%lo")).show()

df_with_schema.printSchema()

df_with_schema.show()





+-------------------+----------------+----------+----------+
|              CodUf|       TipoCrime|Quantidade|      Data|
+-------------------+----------------+----------+----------+
|Rio Grande do Norte|Homicídio doloso|     10242|11/02/2023|
|  Rio Grande do Sul|Homicídio doloso|     16995|16/02/2023|
|     Rio de Janeiro|Homicídio doloso|     31763|22/02/2023|
+-------------------+----------------+----------+----------+

+-------------------+----------------+----------+----------+
|              CodUf|       TipoCrime|Quantidade|      Data|
+-------------------+----------------+----------+----------+
|Rio Grande do Norte|Homicídio doloso|     10242|11/02/2023|
|  Rio Grande do Sul|Homicídio doloso|     16995|16/02/2023|
|     Rio de Janeiro|Homicídio doloso|     31763|22/02/2023|
+-------------------+----------------+----------+----------+

+---------+----------------+----------+----------+
|    CodUf|       TipoCrime|Quantidade|      Data|
+---------+----------------+----------+---

In [24]:

schema = StructType() \
      .add("CodUf",StringType(),True) \
      .add("TipoCrime",StringType(),True) \
      .add("Quantidade",IntegerType(),True) \
      .add("Data",StringType(),True)


df_with_schema = spark.read.options(header='True'
                                    , inferSchema='False'
                                    , delimiter=';'
                                    , encoding="windows-1252") \
                                    .schema(schema) \
                                    .csv(arquivoIndicadorHomicidioRes)

df_with_schema = df_with_schema.filter(df_with_schema['CodUf'].contains('P'))

df_with_schema.printSchema()

df_with_schema.show()

root
 |-- CodUf: string (nullable = true)
 |-- TipoCrime: string (nullable = true)
 |-- Quantidade: integer (nullable = true)
 |-- Data: string (nullable = true)

+----------+----------------+----------+----------+
|     CodUf|       TipoCrime|Quantidade|      Data|
+----------+----------------+----------+----------+
|     Piauí|Homicídio doloso|      4714|04/02/2023|
|   Paraíba|Homicídio doloso|      8902|10/02/2023|
|    Paraná|Homicídio doloso|     15708|15/02/2023|
| São Paulo|Homicídio doloso|     17211|17/02/2023|
|      Pará|Homicídio doloso|     22822|18/02/2023|
|Pernambuco|Homicídio doloso|     28965|21/02/2023|
+----------+----------------+----------+----------+



### 05.2.2 - Semelhante a cláusula LIKE - "INICIA COM" de Bancos de Dados

In [25]:

schema = StructType() \
      .add("CodUf",StringType(),True) \
      .add("TipoCrime",StringType(),True) \
      .add("Quantidade",IntegerType(),True) \
      .add("Data",StringType(),True)


df_with_schema = spark.read.options(header='True'
                                    , inferSchema='False'
                                    , delimiter=';'
                                    , encoding="windows-1252") \
                                    .schema(schema) \
                                    .csv(arquivoIndicadorHomicidioRes)

df_with_schema = df_with_schema.filter(df_with_schema['CodUf'].startswith('R'))

df_with_schema.printSchema()

df_with_schema.show()

root
 |-- CodUf: string (nullable = true)
 |-- TipoCrime: string (nullable = true)
 |-- Quantidade: integer (nullable = true)
 |-- Data: string (nullable = true)

+-------------------+----------------+----------+----------+
|              CodUf|       TipoCrime|Quantidade|      Data|
+-------------------+----------------+----------+----------+
|            Roraima|Homicídio doloso|      1389|28/01/2023|
|           Rondônia|Homicídio doloso|      3482|01/02/2023|
|Rio Grande do Norte|Homicídio doloso|     10242|11/02/2023|
|  Rio Grande do Sul|Homicídio doloso|     16995|16/02/2023|
|     Rio de Janeiro|Homicídio doloso|     31763|22/02/2023|
+-------------------+----------------+----------+----------+



### 05.2.3 - Semelhante a cláusula LIKE - "TERMINA COM" de Bancos de Dados

In [26]:

schema = StructType() \
      .add("CodUf",StringType(),True) \
      .add("TipoCrime",StringType(),True) \
      .add("Quantidade",IntegerType(),True) \
      .add("Data",StringType(),True)


df_with_schema = spark.read.options(header='True'
                                    , inferSchema='False'
                                    , delimiter=';'
                                    , encoding="windows-1252") \
                                    .schema(schema) \
                                    .csv(arquivoIndicadorHomicidioRes)

df_with_schema = df_with_schema.filter(df_with_schema['CodUf'].endswith('o'))

df_with_schema.printSchema()

df_with_schema.show()

root
 |-- CodUf: string (nullable = true)
 |-- TipoCrime: string (nullable = true)
 |-- Quantidade: integer (nullable = true)
 |-- Data: string (nullable = true)

+--------------+----------------+----------+----------+
|         CodUf|       TipoCrime|Quantidade|      Data|
+--------------+----------------+----------+----------+
|   Mato Grosso|Homicídio doloso|      6920|06/02/2023|
|Espírito Santo|Homicídio doloso|      8695|09/02/2023|
|      Maranhão|Homicídio doloso|     13412|13/02/2023|
|     São Paulo|Homicídio doloso|     17211|17/02/2023|
|    Pernambuco|Homicídio doloso|     28965|21/02/2023|
|Rio de Janeiro|Homicídio doloso|     31763|22/02/2023|
+--------------+----------------+----------+----------+



### 05.2.4 - Semelhante a cláusula WHERE de Bancos de Dados

#### WHERE com uma Condição

In [27]:

schema = StructType() \
      .add("CodUf",StringType(),True) \
      .add("TipoCrime",StringType(),True) \
      .add("Quantidade",IntegerType(),True) \
      .add("Data",StringType(),True)


df_with_schema = spark.read.options(header='True'
                                    , inferSchema='False'
                                    , delimiter=';'
                                    , encoding="windows-1252") \
                                    .schema(schema) \
                                    .csv(arquivoIndicadorHomicidioRes)

df_with_schema = df_with_schema.where(df_with_schema['CodUf']=='Rio de Janeiro')

df_with_schema.printSchema()

df_with_schema.show()

root
 |-- CodUf: string (nullable = true)
 |-- TipoCrime: string (nullable = true)
 |-- Quantidade: integer (nullable = true)
 |-- Data: string (nullable = true)

+--------------+----------------+----------+----------+
|         CodUf|       TipoCrime|Quantidade|      Data|
+--------------+----------------+----------+----------+
|Rio de Janeiro|Homicídio doloso|     31763|22/02/2023|
+--------------+----------------+----------+----------+



In [28]:

schema = StructType() \
      .add("CodUf",StringType(),True) \
      .add("TipoCrime",StringType(),True) \
      .add("Quantidade",IntegerType(),True) \
      .add("Data",StringType(),True)

df_with_schema = spark.read.options(header='True'
                                    , inferSchema='False'
                                    , delimiter=';'
                                    , encoding="windows-1252") \
                                    .schema(schema) \
                                    .csv(arquivoIndicadorHomicidioRes)

df_with_schema1 = df_with_schema.where(df_with_schema['Quantidade'] > 10000)

df_with_schema2 = df_with_schema.where(df_with_schema['Quantidade'] >= 10000)

df_with_schema3 = df_with_schema.where(df_with_schema['Quantidade'] != 10000)

df_with_schema4 = df_with_schema.where(df_with_schema['Quantidade'] < 10000)

df_with_schema5 = df_with_schema.where(df_with_schema['Quantidade'] <= 10000)

#df_with_schema.printSchema()

print('--------------------------- MAIOR QUE 10000 ---------------------------')

df_with_schema1.show()

print('--------------------------- MAIOR IGUAL QUE 10000 ---------------------------')

df_with_schema2.show()

print('--------------------------- DIFERENTE DE 10000 ---------------------------')

df_with_schema3.show()

print('--------------------------- MENOR QUE 10000 ---------------------------')

df_with_schema4.show()

print('--------------------------- MENOR IGUAL QUE 10000 ---------------------------')

df_with_schema5.show()

--------------------------- MAIOR QUE 10000 ---------------------------
+-------------------+----------------+----------+----------+
|              CodUf|       TipoCrime|Quantidade|      Data|
+-------------------+----------------+----------+----------+
|Rio Grande do Norte|Homicídio doloso|     10242|11/02/2023|
|            Alagoas|Homicídio doloso|     10267|12/02/2023|
|           Maranhão|Homicídio doloso|     13412|13/02/2023|
|              Goiás|Homicídio doloso|     14512|      null|
|             Paraná|Homicídio doloso|     15708|15/02/2023|
|  Rio Grande do Sul|Homicídio doloso|     16995|16/02/2023|
|          São Paulo|Homicídio doloso|     17211|17/02/2023|
|               Pará|Homicídio doloso|     22822|18/02/2023|
|       Minas Gerais|Homicídio doloso|     24207|19/02/2023|
|              Ceará|Homicídio doloso|     27435|20/02/2023|
|         Pernambuco|Homicídio doloso|     28965|21/02/2023|
|     Rio de Janeiro|Homicídio doloso|     31763|22/02/2023|
|            

#### WHERE com MULTIPLAS Condições

In [29]:

schema = StructType() \
      .add("CodUf",StringType(),True) \
      .add("TipoCrime",StringType(),True) \
      .add("Quantidade",IntegerType(),True) \
      .add("Data",StringType(),True) \


df_with_schema = spark.read.options(header='True'
                                    , inferSchema='False'
                                    , delimiter=';'
                                    , encoding="windows-1252") \
                                    .schema(schema) \
                                    .csv(arquivoIndicadorHomicidioRes)

df_with_schema = df_with_schema.filter(df_with_schema['CodUf'].isin('Rio de Janeiro','São Paulo')) 

df_with_schema.printSchema()

df_with_schema.show()

root
 |-- CodUf: string (nullable = true)
 |-- TipoCrime: string (nullable = true)
 |-- Quantidade: integer (nullable = true)
 |-- Data: string (nullable = true)

+--------------+----------------+----------+----------+
|         CodUf|       TipoCrime|Quantidade|      Data|
+--------------+----------------+----------+----------+
|     São Paulo|Homicídio doloso|     17211|17/02/2023|
|Rio de Janeiro|Homicídio doloso|     31763|22/02/2023|
+--------------+----------------+----------+----------+



#### Quando ocorrer a necessidade de filtrar vários valores na mesma coluna, devemos usaro a cláusula ISIN quando for em campos diferentes devemos usar a cláusula WHERE com MULTIPLAS Condições

#### ISNOTIN

In [30]:

schema = StructType() \
      .add("CodUf",StringType(),True) \
      .add("TipoCrime",StringType(),True) \
      .add("Quantidade",IntegerType(),True) \
      .add("Data",StringType(),True) \


df_with_schema = spark.read.options(header='True'
                                    , inferSchema='False'
                                    , delimiter=';'
                                    , encoding="windows-1252") \
                                    .schema(schema) \
                                    .csv(arquivoIndicadorHomicidioRes)

#df_with_schema = df_with_schema.where(~df_with_schema['CodUf'].isin('Rio de Janeiro','São Paulo'))

df_with_schema = df_with_schema.where(df_with_schema['CodUf'].isin('Rio de Janeiro','São Paulo')==False)

df_with_schema.printSchema()

df_with_schema.show()



root
 |-- CodUf: string (nullable = true)
 |-- TipoCrime: string (nullable = true)
 |-- Quantidade: integer (nullable = true)
 |-- Data: string (nullable = true)

+-------------------+----------------+----------+----------+
|              CodUf|       TipoCrime|Quantidade|      Data|
+-------------------+----------------+----------+----------+
|            Roraima|Homicídio doloso|      1389|28/01/2023|
|              Amapá|Homicídio doloso|      1948|29/01/2023|
|               Acre|Homicídio doloso|      2007|30/01/2023|
|          Tocantins|Homicídio doloso|      null|31/01/2023|
|           Rondônia|Homicídio doloso|      3482|01/02/2023|
|   Distrito Federal|Homicídio doloso|      3518|02/02/2023|
| Mato Grosso do Sul|Homicídio doloso|      3905|03/02/2023|
|              Piauí|Homicídio doloso|      4714|04/02/2023|
|     Santa Catarina|Homicídio doloso|      5832|05/02/2023|
|        Mato Grosso|Homicídio doloso|      6920|06/02/2023|
|            Sergipe|Homicídio doloso|      

### 05.2.5 - Semelhante a cláusula IN de Bancos de Dados

In [31]:

schema = StructType() \
      .add("CodUf",StringType(),True) \
      .add("TipoCrime",StringType(),True) \
      .add("Quantidade",IntegerType(),True) \
      .add("Data",StringType(),True) \


df_with_schema = spark.read.options(header='True'
                                    , inferSchema='False'
                                    , delimiter=';'
                                    , encoding="windows-1252") \
                                    .schema(schema) \
                                    .csv(arquivoIndicadorHomicidioRes)

df_with_schema = df_with_schema.where(df_with_schema['CodUf'].isin('Rio de Janeiro'))

df_with_schema.printSchema()

df_with_schema.show()

root
 |-- CodUf: string (nullable = true)
 |-- TipoCrime: string (nullable = true)
 |-- Quantidade: integer (nullable = true)
 |-- Data: string (nullable = true)

+--------------+----------------+----------+----------+
|         CodUf|       TipoCrime|Quantidade|      Data|
+--------------+----------------+----------+----------+
|Rio de Janeiro|Homicídio doloso|     31763|22/02/2023|
+--------------+----------------+----------+----------+



In [32]:

schema = StructType() \
      .add("CodUf",StringType(),True) \
      .add("TipoCrime",StringType(),True) \
      .add("Quantidade",IntegerType(),True) \
      .add("Data",StringType(),True) \


df_with_schema = spark.read.options(header='True'
                                    , inferSchema='False'
                                    , delimiter=';'
                                    , encoding="windows-1252") \
                                    .schema(schema) \
                                    .csv(arquivoIndicadorHomicidioRes)

df_with_schema = df_with_schema.where(df_with_schema['CodUf'].isin('Rio de Janeiro','São Paulo'))

df_with_schema = df_with_schema.where(df_with_schema['CodUf'].isin(['Rio de Janeiro','São Paulo']))

df_with_schema.printSchema()

df_with_schema.show()

root
 |-- CodUf: string (nullable = true)
 |-- TipoCrime: string (nullable = true)
 |-- Quantidade: integer (nullable = true)
 |-- Data: string (nullable = true)

+--------------+----------------+----------+----------+
|         CodUf|       TipoCrime|Quantidade|      Data|
+--------------+----------------+----------+----------+
|     São Paulo|Homicídio doloso|     17211|17/02/2023|
|Rio de Janeiro|Homicídio doloso|     31763|22/02/2023|
+--------------+----------------+----------+----------+



## 05.3 - Tratando Valores com UPPER, LOWER e INITCAPT

In [33]:

schema = StructType() \
      .add("CodUf",StringType(),True) \
      .add("TipoCrime",StringType(),True) \
      .add("Quantidade",IntegerType(),True) \
      .add("Data",StringType(),True) \


df_with_schema = spark.read.options(header='True'
                                    , inferSchema='False'
                                    , delimiter=';'
                                    , encoding="windows-1252") \
                                    .schema(schema) \
                                    .csv(arquivoIndicadorHomicidioRes)

df_with_schema = df_with_schema.withColumn("CodUf_upper", upper(col("CodUf")))

df_with_schema = df_with_schema.withColumn("CodUf_lower", lower(col("CodUf")))

df_with_schema = df_with_schema.withColumn("CodUf_incap", initcap(col("CodUf")))

df_with_schema.printSchema()

df_with_schema.show()


root
 |-- CodUf: string (nullable = true)
 |-- TipoCrime: string (nullable = true)
 |-- Quantidade: integer (nullable = true)
 |-- Data: string (nullable = true)
 |-- CodUf_upper: string (nullable = true)
 |-- CodUf_lower: string (nullable = true)
 |-- CodUf_incap: string (nullable = true)

+-------------------+----------------+----------+----------+-------------------+-------------------+-------------------+
|              CodUf|       TipoCrime|Quantidade|      Data|        CodUf_upper|        CodUf_lower|        CodUf_incap|
+-------------------+----------------+----------+----------+-------------------+-------------------+-------------------+
|            Roraima|Homicídio doloso|      1389|28/01/2023|            RORAIMA|            roraima|            Roraima|
|              Amapá|Homicídio doloso|      1948|29/01/2023|              AMAPÁ|              amapá|              Amapá|
|               Acre|Homicídio doloso|      2007|30/01/2023|               ACRE|               acre|   

# 05 - Trabalhando com Arquivos Parquet

In [34]:
#arquivoDestino = '../ProjetoSparkKB/Dados/Parquet/indicador_homocidio.parquet'

#df_with_schema3.write.mode("overwrite").parquet(arquivoDestino)

# 06 - Transformando o DataFrame Spark em Tabela Temporária

In [35]:
df.createOrReplaceTempView('teste')

## 06.1 - Lendo a Tabela e Transformando em outro DataFrame Spark

In [36]:
df2 = spark.sql("""
SELECT DT_GERACAO 
FROM TESTE
""")

# 07 - Convertendo DataType de um DataFrame Spark

## 07.1 -  De uma Coluna 

In [37]:
df2.withColumn("DT_GERACAO", df2["DT_GERACAO"].cast("timestamp")).printSchema()

root
 |-- DT_GERACAO: timestamp (nullable = true)



## 07.2 - Criando uma nova Coluna

In [38]:
df2.withColumn("DTGERACAO", df2["DT_GERACAO"].cast("timestamp")).printSchema()

root
 |-- DT_GERACAO: string (nullable = true)
 |-- DTGERACAO: timestamp (nullable = true)



# 08 - Renomeando Coluna(s) em DataFrame Spark

## 08.1 - Uma Única Coluna

In [39]:
df2.withColumnRenamed("DT_GERACAO","DT_GERACAO2").printSchema()

root
 |-- DT_GERACAO2: string (nullable = true)



## 08.2 - Várias Colunas

In [40]:
df3 = spark.sql("""
SELECT DT_GERACAO , HH_GERACAO
FROM TESTE
""")

df3 = df3.withColumnRenamed("DT_GERACAO","DtGeracao") \
            .withColumnRenamed("HH_GERACAO","HrGeracao")
df3.printSchema()


root
 |-- DtGeracao: string (nullable = true)
 |-- HrGeracao: string (nullable = true)



# 09 - Ordenando um DataFrame Spark

## 09.1 - Ordenando um DataFrame Spark por uma Coluna

In [41]:
df4 = spark.sql("""
SELECT SG_UF , NM_URNA_CANDIDATO
FROM TESTE
""")

df4.show()

df4.sort("SG_UF").show(truncate=False)


+-----+--------------------+
|SG_UF|   NM_URNA_CANDIDATO|
+-----+--------------------+
|   CE|    TONY IN�CIO SHOW|
|   RJ|ENFERMEIRA SONIA ...|
|   RS|       TANGER JARDIM|
|   SP|  VAVA DO TRANSPORTE|
|   AM|           LEDA MAIA|
|   AL|        �TALO BASTOS|
|   MG|     CARCILEI SOARES|
|   RJ|PAI MARCELO DE OX...|
|   PR|            DR MAGNO|
|   ES|       VANETE CARIRI|
|   PR|       RONNIE CASTRO|
|   PA|      ELIEL FAUSTINO|
|   RJ|        MARCOS UCH�A|
|   ES|IDELFONSO DA VITORIA|
|   AP|PAULO CUNHA COLET...|
|   SE|GILENO PAO FILHO ...|
|   AM|      DR GEORGE LINS|
|   AP|      ALDILENE SOUZA|
|   BA|MISSIONARIA SIMON...|
|   RJ|KARLA DE LUCAS DO...|
+-----+--------------------+
only showing top 20 rows

+-----+---------------------+
|SG_UF|NM_URNA_CANDIDATO    |
+-----+---------------------+
|AC   |PENINHA MAGALHAES    |
|AC   |ANGELINO DO PANDEIRO |
|AC   |TEREZINHA ARAUJO     |
|AC   |HENRY NOGUEIRA       |
|AC   |JOYCE DOS BURACOS    |
|AC   |ADAL�CIO J�NIOR      |
|AC   |T

## 09.2 - Ordenando um DataFrame Spark por Várias Colunas

In [42]:
df4.sort("SG_UF","NM_URNA_CANDIDATO").show(truncate=False)


+-----+---------------------+
|SG_UF|NM_URNA_CANDIDATO    |
+-----+---------------------+
|AC   |ADAILTON CRUZ        |
|AC   |ADALBERTO ALMEIDA    |
|AC   |ADAL�CIO J�NIOR      |
|AC   |ADEKSON LIMA         |
|AC   |ADENILSON DUAM       |
|AC   |ADERL�NDIO FRAN�A    |
|AC   |ADEVILSON QUEIROZ    |
|AC   |ADONAY               |
|AC   |ADRIANA ANDRADE      |
|AC   |ADRIANA MAIA         |
|AC   |ADRIANA MELO         |
|AC   |AFONSO CONTADOR      |
|AC   |AFONSO FERNANDES     |
|AC   |ALAN RICK            |
|AC   |ALDA RADINE          |
|AC   |ALDENORA SALES       |
|AC   |ALEXANDRE FERNANDES  |
|AC   |ALEXANDRINA VASQUES  |
|AC   |ALMINERES DO ELDORADO|
|AC   |AMANDA SOUZA         |
+-----+---------------------+
only showing top 20 rows



In [43]:

df4.sort(col("SG_UF"),col("NM_URNA_CANDIDATO")).show(truncate=False)

+-----+---------------------+
|SG_UF|NM_URNA_CANDIDATO    |
+-----+---------------------+
|AC   |ADAILTON CRUZ        |
|AC   |ADALBERTO ALMEIDA    |
|AC   |ADAL�CIO J�NIOR      |
|AC   |ADEKSON LIMA         |
|AC   |ADENILSON DUAM       |
|AC   |ADERL�NDIO FRAN�A    |
|AC   |ADEVILSON QUEIROZ    |
|AC   |ADONAY               |
|AC   |ADRIANA ANDRADE      |
|AC   |ADRIANA MAIA         |
|AC   |ADRIANA MELO         |
|AC   |AFONSO CONTADOR      |
|AC   |AFONSO FERNANDES     |
|AC   |ALAN RICK            |
|AC   |ALDA RADINE          |
|AC   |ALDENORA SALES       |
|AC   |ALEXANDRE FERNANDES  |
|AC   |ALEXANDRINA VASQUES  |
|AC   |ALMINERES DO ELDORADO|
|AC   |AMANDA SOUZA         |
+-----+---------------------+
only showing top 20 rows



# 10 - Agrupando um DataFrame Spark

In [44]:
df4.groupBy("SG_UF").count().show(truncate=False)

+-----+-----+
|SG_UF|count|
+-----+-----+
|SC   |996  |
|RO   |633  |
|PI   |433  |
|AM   |664  |
|RR   |612  |
|GO   |1243 |
|TO   |531  |
|MT   |525  |
|BR   |30   |
|SP   |3659 |
|ES   |782  |
|PB   |756  |
|RS   |1437 |
|MS   |598  |
|AL   |503  |
|MG   |2562 |
|PA   |1043 |
|BA   |1723 |
|SE   |545  |
|PE   |1142 |
+-----+-----+
only showing top 20 rows



# 11 - Funções de Agrupamento (Count(), Sum(), Min(), Max() e Mean())

In [45]:
df5 = spark.sql("""
SELECT SG_UF , NM_URNA_CANDIDATO, SQ_CANDIDATO
FROM TESTE
""")

df5.groupBy("SG_UF") \
    .agg(count("SQ_CANDIDATO").alias("count_SQ_CANDIDATO"), \
         sum("SQ_CANDIDATO").alias("sum_SQ_CANDIDATO"), \
         avg("SQ_CANDIDATO").alias("avg_SQ_CANDIDATO"), \
         min("SQ_CANDIDATO").alias("min_SQ_CANDIDATO"), \
         max("SQ_CANDIDATO").alias("max_SQ_CANDIDATO") \
     ) \
    .show(truncate=False)

+-----+------------------+-------------------+---------------------+----------------+----------------+
|SG_UF|count_SQ_CANDIDATO|sum_SQ_CANDIDATO   |avg_SQ_CANDIDATO     |min_SQ_CANDIDATO|max_SQ_CANDIDATO|
+-----+------------------+-------------------+---------------------+----------------+----------------+
|AC   |542               |5.420889490165E12  |1.0001641125765682E10|10001595335     |10001739225     |
|AL   |503               |1.0060834407453E13 |2.0001658861735588E10|20001605776     |20001739199     |
|AM   |664               |2.6561091537892E13 |4.000164388236747E10 |40001596222     |40001739338     |
|AP   |557               |1.6710907935804E13 |3.000163004632675E10 |30001595385     |30001739811     |
|BA   |1723              |8.6152807743451E13 |5.0001629566715614E10|50001596234     |50001739212     |
|BR   |30                |8.400048796108E12  |2.8000162653693335E11|280001600166    |280001735149    |
|CE   |1018              |6.1081667260283E13 |6.00016377802387E10  |60001